In [1]:
!nvidia-smi

Thu Apr  8 09:21:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   56C    P0   171W / 250W |  32322MiB / 32480MiB |     99%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [1]:
import torch
import random
import pandas as pd
import numpy as np
import os
from glob import glob
import shutil
from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from RandAugment import RandAugment
import torch.optim as optim
import sys
import time
from efficientnet_pytorch import EfficientNet

from PIL import Image
from tqdm import tqdm
import timm
from pprint import pprint

In [2]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  
    
seed_everything()

In [3]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [4]:
test_path = './input/data/eval/images/'
df_test = pd.read_csv('./input/data/eval/info.csv')

test_img_paths = []
    
for path in df_test['ImageID'].tolist():
    test_img_paths.append(test_path+path)

In [5]:
test_transforms = transforms.Compose([transforms.CenterCrop((300, 230)),
                                      transforms.Resize((224, 224)),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                                     ])
test_transforms2 = transforms.Compose([
                                      transforms.Resize((224, 224), Image.BILINEAR),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246)),
                                     ])

test_dataset = TestDataset(test_img_paths, transform=test_transforms) 
test_dataset2 = TestDataset(test_img_paths, transform=test_transforms2) 

In [6]:
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=2)
# test_loader2 = DataLoader(test_dataset2, batch_size=256, shuffle=False, num_workers=1)

# 각 Task Model Load

In [7]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
    
RANDOM_SEED = 1
BATCH_SIZE = 256
seed_everything(RANDOM_SEED)

In [8]:
## Multi Model
lr = 7e-5

# ModelA = mask, ModelB = age, ModelC = gender
modelA = EfficientNet.from_pretrained('efficientnet-b4', num_classes = 3)
modelA_1 = EfficientNet.from_pretrained('efficientnet-b4', num_classes = 3)
modelA_2 = EfficientNet.from_pretrained('efficientnet-b4', num_classes = 3)
modelB_1 = EfficientNet.from_pretrained('efficientnet-b4', num_classes = 3)
modelB_2 = EfficientNet.from_pretrained('efficientnet-b4', num_classes = 3)
modelB_3 = EfficientNet.from_pretrained('efficientnet-b4', num_classes = 3)
modelB_4 = EfficientNet.from_pretrained('efficientnet-b4', num_classes = 3)
modelB_5 = EfficientNet.from_pretrained('efficientnet-b4', num_classes = 3)
modelC = EfficientNet.from_pretrained('efficientnet-b4', num_classes = 2)
modelC_1 = EfficientNet.from_pretrained('efficientnet-b4', num_classes = 2)
# modelC_2 = EfficientNet.from_pretrained('efficientnet-b4', num_classes = 2)

# modelA._fc = nn.Linear(1792, 3)
# modelB_1._fc = nn.Linear(1792, 3)
# modelB_2._fc = nn.Linear(1792, 3)
# modelB_3._fc = nn.Linear(1792, 3)
# modelC._fc = nn.Linear(1792, 2)

PATH_A = './checkpoint/mask_model(eff_b4(n737))/best_model.pt'
PATH_A_1 = './checkpoint/mask_model(eff_b4(n1062))/best_model.pt'
PATH_A_2 = './checkpoint/mask_model(eff_b4(noR_n737))/best_model.pt'
PATH_B_1 = './checkpoint/age_model(eff_b4_(noR_n0))/best_model.pt'
PATH_B_2 = './checkpoint/age_model(eff_b4_softlabe(n1062 s1))/best_model.pt'  ##
PATH_B_3 = './checkpoint/age_model(eff_b4_(noR_n1062))/best_model.pt'
PATH_B_4 = './checkpoint/age_model(eff_b4_softlabel2)/best_model.pt'  ##
PATH_B_5 = './checkpoint/age_model(eff_b4_(noR_n1400))/best_model.pt'  ##
PATH_C = './checkpoint/gender_model(eff_b4)/best_model_EPOCH190.pt'
PATH_C_1 = './checkpoint/gender_model(eff_b4_n1062)/best_model.pt'
# PATH_C_2 = './checkpoint/gender_model(eff_b4)/best_model_EPOCH147.pt'

checkpointA = torch.load(PATH_A)
checkpointA_1 = torch.load(PATH_A_1)
checkpointA_2 = torch.load(PATH_A_2)
checkpointB_1 = torch.load(PATH_B_1)
checkpointB_2 = torch.load(PATH_B_2)
checkpointB_3 = torch.load(PATH_B_3)
checkpointB_4 = torch.load(PATH_B_4)
checkpointB_5 = torch.load(PATH_B_5)
checkpointC = torch.load(PATH_C)
checkpointC_1 = torch.load(PATH_C_1)
checkpointC_2 = torch.load(PATH_C_1)

modelA.load_state_dict(checkpointA)
modelA.cuda()
modelA_1.load_state_dict(checkpointA_1)
modelA_1.cuda()
modelA_2.load_state_dict(checkpointA_2)
modelA_2.cuda()
modelB_1.load_state_dict(checkpointB_1)
modelB_1.cuda()
modelB_2.load_state_dict(checkpointB_2)
modelB_2.cuda()
modelB_3.load_state_dict(checkpointB_3)
modelB_3.cuda()
modelB_4.load_state_dict(checkpointB_4)
modelB_4.cuda()
modelB_5.load_state_dict(checkpointB_5)
modelB_5.cuda()
modelC.load_state_dict(checkpointC)
modelC.cuda()
modelC_1.load_state_dict(checkpointC_1)
modelC_1.cuda()
# modelC_2.load_state_dict(checkpointC_2)
# modelC_2.cuda()

# Freeze the feature extracting convolution layers
for param in modelA.parameters():
    param.requires_grad = False
for param in modelA_1.parameters():
    param.requires_grad = False
for param in modelA_2.parameters():
    param.requires_grad = False
for param in modelB_1.parameters():
    param.requires_grad = False
for param in modelB_2.parameters():
    param.requires_grad = False
for param in modelB_3.parameters():
    param.requires_grad = False
for param in modelB_4.parameters():
    param.requires_grad = False
for param in modelB_5.parameters():
    param.requires_grad = False
for param in modelC.parameters():
    param.requires_grad = False
for param in modelC_1.parameters():
    param.requires_grad = False
# for param in modelC_2.parameters():
#     param.requires_grad = False
    


Loaded pretrained weights for efficientnet-b4
Loaded pretrained weights for efficientnet-b4
Loaded pretrained weights for efficientnet-b4
Loaded pretrained weights for efficientnet-b4
Loaded pretrained weights for efficientnet-b4
Loaded pretrained weights for efficientnet-b4
Loaded pretrained weights for efficientnet-b4
Loaded pretrained weights for efficientnet-b4
Loaded pretrained weights for efficientnet-b4
Loaded pretrained weights for efficientnet-b4


In [9]:
# ## Multi Model
# lr = 7e-5

# # ModelA = mask, ModelB = age, ModelC = gender
# modelA = timm.create_model('ecaresnet50d_pruned', pretrained=True)
# modelB = EfficientNet.from_pretrained('efficientnet-b4')
# modelC = timm.create_model('ecaresnet50d_pruned', pretrained=True)

# modelA.fc = nn.Linear(2022, 3)
# modelB._fc = nn.Linear(1792, 3)
# modelC.fc = nn.Linear(2022, 2)

# PATH_A = './checkpoint/mask_model/best_model_EPOCH8.pt'
# PATH_B = './checkpoint/age_model(eff_b4_softlabel2)/best_model.pt'
# PATH_C = './checkpoint/gender_model/best_model_EPOCH34.pt'

# checkpointA = torch.load(PATH_A)
# checkpointB = torch.load(PATH_B)
# checkpointC = torch.load(PATH_C)

# modelA.load_state_dict(checkpointA)
# modelA.cuda()
# modelB.load_state_dict(checkpointB)
# modelB.cuda()
# modelC.load_state_dict(checkpointC)
# modelC.cuda()

# # Freeze the feature extracting convolution layers
# for param in modelA.parameters():
#     param.requires_grad = False
# for param in modelB.parameters():
#     param.requires_grad = False
# for param in modelC.parameters():
#     param.requires_grad = False

In [12]:
class MuliModel(nn.Module):
    def __init__(self, modelA, modelA_1, modelA_2, modelB_1, modelB_2, modelB_3, modelB_4, modelB_5, modelC, modelC_1):
        super(MuliModel, self).__init__()
        self.relu = nn.ReLU(inplace=True)

        self.modelA = modelA
        self.modelA_1 = modelA_1
        self.modelA_2 = modelA_2
        self.modelB_1 = modelB_1
        self.modelB_2 = modelB_2
        self.modelB_3 = modelB_3
        self.modelB_4 = modelB_4
        self.modelB_5 = modelB_5
        self.modelC = modelC
        self.modelC_1 = modelC_1
#         self.modelC_2 = modelC_2
        
        self.softmax = nn.Softmax(dim=1)
        self.FC_layer = nn.Linear(8, 18)
        self.dropout = nn.Dropout()
        self.classifier = nn.Linear(18, 18)
        
    def forward(self, x):
        
        x1 = self.softmax(self.modelA(x))
        x1_1 = self.softmax(self.modelA_1(x))
        x1_2 = self.softmax(self.modelA_2(x))
        
        x2_1 = self.softmax(self.modelB_1(x))
        x2_2 = self.softmax(self.modelB_2(x))
        x2_3 = self.softmax(self.modelB_3(x))
        x2_4 = self.softmax(self.modelB_4(x))
        x2_5 = self.softmax(self.modelB_5(x))
        
        x3_1 = self.softmax(self.modelC(x))
        x3_2 = self.softmax(self.modelC_1(x))
#         x3_3 = self.softmax(self.modelC_2(x))
        
        x1 = torch.argmax(x1 * 0.6 + x1_2 * 0.6 + x1_2 * 1.2, 1)
        x2 = torch.argmax(x2_1 + x2_2 * 0.3 + x2_3 + x2_4 * 0.3 + x2_5, 1)
        x3 = torch.argmax(x3_1 + x3_2, 1)
        label_pred = x1 * 6 + x3 * 3 + x2        
        return label_pred

model = MuliModel(modelA, modelA_1, modelA_2, modelB_1, modelB_2, modelB_3, modelB_4, modelB_5, modelC, modelC_1)

In [13]:
test_dir = '/opt/ml/input/data/eval'

# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv('./input/data/eval/info.csv')
loader = test_loader

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
model = model.to(DEVICE)
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(DEVICE)
        pred = model(images)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv('./submission.csv', index=False)
print('test inference is done!')

test inference is done!


In [2]:
import torch

In [3]:
a = torch.tensor([[1],[1],[0],[1],[0]])
b = torch.tensor([[2],[3],[1],[1],[0]])

In [4]:
torch.cat((a, b), 1)

tensor([[1, 2],
        [1, 3],
        [0, 1],
        [1, 1],
        [0, 0]])

# Augmentation 다르게 Ensemble

In [8]:
from importlib import import_module

In [9]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
    
RANDOM_SEED = 42
BATCH_SIZE = 256
seed_everything(RANDOM_SEED)

In [76]:
model_cls = getattr(import_module("baseline.model"), 'Efficientnet_b4')
modelD = model_cls(num_classes = 3)
PATH_D = './baseline/model/Efficientnet_b4_adam_lr7e5_resi244/best.pth'
checkpointD = torch.load(PATH_D)
modelD.load_state_dict(checkpointD)
modelD.cuda()
for param in modelD.parameters():
    param.requires_grad = False
    
model_cls = getattr(import_module("baseline.model"), 'Efficientnet_b4')
modelE = model_cls(num_classes = 3)
PATH_E = './baseline/model/Efficientnet_b4_adam_lr8e5_resi244/best.pth'
checkpointE = torch.load(PATH_E)
modelE.load_state_dict(checkpointE)
modelE.cuda()
for param in modelE.parameters():
    param.requires_grad = False


Loaded pretrained weights for efficientnet-b4
Loaded pretrained weights for efficientnet-b4


In [79]:
model_cls = getattr(import_module("baseline.model"), 'Efficientnet_b4')
modelB_1 = model_cls(num_classes = 3)
PATH_B_1 = './baseline/model/Efficientnet_b4_adam_lr6e5_resi244/best.pth'
checkpointB_1 = torch.load(PATH_B_1)
modelB_1.load_state_dict(checkpointB_1)
modelB_1.cuda()
for param in modelB_1.parameters():
    param.requires_grad = False

Loaded pretrained weights for efficientnet-b4


In [19]:


model_cls = getattr(import_module("baseline.model"), 'Efficientnet_b4')
modelF = model_cls(num_classes = 3)
PATH_F = './baseline/model/Efficientnet_b4_adam_lr6e5_okay/best.pth'
checkpointF = torch.load(PATH_F)
modelF.load_state_dict(checkpointF)
modelF.cuda()
for param in modelF.parameters():
    param.requires_grad = False

Loaded pretrained weights for efficientnet-b4


In [10]:
## Multi Model
lr = 7e-5
model_cls = getattr(import_module("baseline.model"), 'Efficientnet_b4')
# ModelA = mask, ModelB = age, ModelC = gender
modelA = EfficientNet.from_pretrained('efficientnet-b4')
modelB_1 = model_cls(num_classes = 3)
modelB_2 = model_cls(num_classes = 3)
modelB_3 = model_cls(num_classes = 3)
modelC = EfficientNet.from_pretrained('efficientnet-b4')

modelA._fc = nn.Linear(1792, 3)

modelC._fc = nn.Linear(1792, 2)

PATH_A = './checkpoint/mask_model(eff_b4(n737))/best_model.pt'
PATH_B_1 = './baseline/model/Efficientnet_b4_adam_lr6e5_resi244/best.pth'
PATH_B_2 = './baseline/model/Efficientnet_b4_adam_lr7e5_resi244/best.pth'
PATH_B_3 = './baseline/model/Efficientnet_b4_adam_lr8e5_resi244/best.pth' 
PATH_C = './checkpoint/gender_model(eff_b4)/best_model_EPOCH190.pt'

checkpointA = torch.load(PATH_A)
checkpointB_1 = torch.load(PATH_B_1)
checkpointB_2 = torch.load(PATH_B_2)
checkpointB_3 = torch.load(PATH_B_3)
checkpointC = torch.load(PATH_C)

modelA.load_state_dict(checkpointA)
modelA.cuda()
modelB_1.load_state_dict(checkpointB_1)
modelB_1.cuda()
modelB_2.load_state_dict(checkpointB_2)
modelB_2.cuda()
modelB_3.load_state_dict(checkpointB_3)
modelB_3.cuda()
modelC.load_state_dict(checkpointC)
modelC.cuda()

# Freeze the feature extracting convolution layers
for param in modelA.parameters():
    param.requires_grad = False
for param in modelB_1.parameters():
    param.requires_grad = False
for param in modelB_2.parameters():
    param.requires_grad = False
for param in modelB_3.parameters():
    param.requires_grad = False
for param in modelC.parameters():
    param.requires_grad = False

Loaded pretrained weights for efficientnet-b4
Loaded pretrained weights for efficientnet-b4
Loaded pretrained weights for efficientnet-b4
Loaded pretrained weights for efficientnet-b4
Loaded pretrained weights for efficientnet-b4


In [11]:
# 왜인지 모르겠지만 eval은 하면 안되는것 갇음

class AgeModel(nn.Module):
    def __init__(self, modelB_1, modelB_2, modelB_3):
        super(AgeModel, self).__init__()
        self.relu = nn.ReLU(inplace=True)

        self.modelB_1 = modelB_1
        self.modelB_2 = modelB_2
        self.modelB_3 = modelB_3
        
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x2_1 = self.softmax(self.modelB_1(x))
        x2_2 = self.softmax(self.modelB_2(x))
        x2_3 = self.softmax(self.modelB_3(x))
        
        x2 = torch.argmax(x2_1 + x2_2 + x2_3, 1)
#         x2 = torch.argmax(x2_2 + x2_3, 1)
        return x2

model_Age = AgeModel(modelB_1, modelB_2, modelB_3)

In [12]:
class GenderMaskModel(nn.Module):
    def __init__(self, modelA, modelC):
        super(GenderMaskModel, self).__init__()
        self.relu = nn.ReLU(inplace=True)

        self.modelA = modelA
        self.modelC = modelC
        
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x1 = torch.argmax(self.modelA(x), 1)
        x3 = torch.argmax(self.modelC(x), 1)
        label_pred = x1 * 6 + x3 * 3   
        return label_pred

model_MaskGender = GenderMaskModel(modelA, modelC)

In [13]:
test_dir = '/opt/ml/input/data/eval'

# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv('./input/data/eval/info.csv')
loader_age = test_loader2
loader_MaskGender = test_loader

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
model_Age = model_Age.to(DEVICE)
# model_Age.eval()

model_MaskGender = model_MaskGender.to(DEVICE)
model_MaskGender.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
# all_predictions = []
age_predictions = []
maskgeder_predictions = []
for images in loader_age:
    with torch.no_grad():
        images = images.to(DEVICE)
        pred = model_Age(images)
        age_predictions.extend(pred.cpu().numpy())

print("Age predict complite")
        
for images in loader_MaskGender:
    with torch.no_grad():
        images = images.to(DEVICE)
        pred = model_MaskGender(images)
        maskgeder_predictions.extend(pred.cpu().numpy())

print("Mask and Gender predict complite")        

all_predictions = np.array(age_predictions) + np.array(maskgeder_predictions)
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join('./', 'submission.csv'), index=False)
print('test inference is done!')

Age predict complite
Mask and Gender predict complite
test inference is done!


In [22]:
len(age_predictions)

12600

In [23]:
len(maskgeder_predictions)

12600

In [24]:
len(all_predictions)

12600

In [25]:
all_predictions[0]

14

In [26]:
age_predictions[0]

2

In [27]:
maskgeder_predictions[0]

12

In [31]:
from baseline.dataset import *

In [36]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [37]:
dataset = MaskSplitByProfileDataset(data_dir='/opt/ml/input/data/train/images',
                                   
        n=0,
        k=1,
        val_ratio=0.2)

In [35]:
num_classes = 3

In [43]:
transform = BaseAugmentation(
    resize=(244, 244),
    mean=dataset.mean,
    std=dataset.std,
)

transform_val = BaseAugmentation(
    resize=(244, 244),
    mean=dataset.mean,
    std=dataset.std,
)
dataset.set_transform(transform, transform_val)

# -- data_loader
train_set, val_set = dataset.split_dataset()

train_loader = DataLoader(
    train_set,
    batch_size=64,
    num_workers=8,
    shuffle=False,
    pin_memory=use_cuda,
    drop_last=True,
)

val_loader = DataLoader(
    val_set,
    batch_size=256,
    num_workers=1,
    shuffle=False,
    pin_memory=use_cuda,
    drop_last=False,
)

In [41]:
modelB_1

Efficientnet_b4(
  (model): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
    )
    (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          48, 12, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          12, 48, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
      

In [46]:
modelB_1.eval()
with torch.no_grad():
    print("Calculating validation results...")
    modelB_1.eval()
    val_loss_items = []
    val_acc_items = []
    val_f1_pred = []
    val_f1_y = []
    acc_okay = 0
    count_all = 0
    for val_batch in val_loader:
        inputs, labels, levels = val_batch
        inputs = inputs.to(device)
        labels = labels.to(device)
        levels = levels.to(device)

        outs = modelB_1(inputs)
        preds = torch.argmax(outs, dim=-1)

        acc_item = (labels == preds).sum().item()

        val_f1_pred += preds.tolist()
        val_f1_y += labels.tolist()

        val_acc_items.append(acc_item)
        acc_okay += acc_item
        count_all += len(preds)
        break

Calculating validation results...


In [47]:
dataset

(tensor([[[-0.0288, -0.0122, -0.0122,  ..., -0.5252, -0.5252, -0.5583],
          [-0.0122, -0.0122,  0.0043,  ..., -0.5086, -0.5252, -0.5583],
          [ 0.0043,  0.0043,  0.0043,  ..., -0.4590, -0.4590, -0.4590],
          ...,
          [-1.2036, -1.2863, -1.2532,  ..., -1.0712, -1.2202, -1.8158],
          [-1.7331, -1.5676, -1.4684,  ..., -0.7734, -0.9720, -1.3856],
          [-1.9151, -1.8655, -1.9648,  ..., -0.7568, -1.0547, -1.0381]],
 
         [[ 0.4363,  0.4522,  0.4522,  ..., -0.8815, -0.8815, -0.9132],
          [ 0.4522,  0.4522,  0.4680,  ..., -0.8656, -0.8815, -0.9132],
          [ 0.4680,  0.4680,  0.4680,  ..., -0.8180, -0.8180, -0.8180],
          ...,
          [-0.9609, -1.0244, -0.9609,  ..., -0.7703, -0.9450, -1.5483],
          [-1.5801, -1.3895, -1.2466,  ..., -0.4846, -0.7068, -1.1355],
          [-1.8341, -1.7706, -1.8023,  ..., -0.4687, -0.7862, -0.7862]],
 
         [[ 0.7310,  0.7469,  0.7469,  ..., -1.4530, -1.4530, -1.4849],
          [ 0.7469,  0.7469,

In [49]:
ind = []
for phase, indices in MaskSplitByProfileDataset.indices.items():
    ind = indices
    if pahse == "val":
        break

AttributeError: type object 'MaskSplitByProfileDataset' has no attribute 'indices'

In [60]:
a = nn.Softmax(dim=1)

In [61]:
b = torch.tensor([[1,2,3],[2,-1,0.8], [1,-9,-10],[0,0.1,0.9]])

In [63]:
c = torch.tensor([[0,4,2],[1,-1,9], [9,-9,-10],[2,0.1,0.9]])

In [64]:
a(b) + a(c)

tensor([[1.0591e-01, 1.1115e+00, 7.8255e-01],
        [7.4054e-01, 3.6898e-02, 1.2226e+00],
        [1.9999e+00, 4.5412e-05, 1.6706e-05],
        [8.9363e-01, 3.4300e-01, 7.6337e-01]])

In [65]:
a(b)

tensor([[9.0031e-02, 2.4473e-01, 6.6524e-01],
        [7.4020e-01, 3.6853e-02, 2.2294e-01],
        [9.9994e-01, 4.5397e-05, 1.6701e-05],
        [2.1907e-01, 2.4211e-01, 5.3882e-01]])

In [66]:
a(c)

tensor([[1.5876e-02, 8.6681e-01, 1.1731e-01],
        [3.3533e-04, 4.5383e-05, 9.9962e-01],
        [1.0000e+00, 1.5230e-08, 5.6028e-09],
        [6.7456e-01, 1.0089e-01, 2.2454e-01]])

In [71]:
torch.argmax(a(b) + a(c), 1)

tensor([1, 2, 0, 0])

In [33]:
test_dir = '/opt/ml/input/data/eval'

# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv('./input/data/eval/info.csv')
loader_age = test_loader2
loader_age2 = test_loader2

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
model_Age = model_Age.to(DEVICE)
model_Age.eval()

modelB_1 = modelB_1.to(DEVICE)
modelB_1.eval()

# modelD = modelD.to(DEVICE)
# modelD.eval()

# modelE = modelE.to(DEVICE)
# modelE.eval()

modelF = modelF.to(DEVICE)
modelF.eval()


# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
# all_predictions = []
age_predictions1 = []
age_predictions2 = []
age_predictions3 = []
age_predictions4 = []
age_predictions5 = []

for images in loader_age:
    with torch.no_grad():
        images = images.to(DEVICE)
        pred = model_Age(images)
        age_predictions1.extend(pred.cpu().numpy())
        pred = modelB_1(images)
        pred = torch.argmax(pred,1)
        age_predictions2.extend(pred.cpu().numpy())
#         pred = modelD(images)
#         pred = torch.argmax(pred,1)
#         age_predictions3.extend(pred.cpu().numpy())
#         pred = modelE(images)
#         pred = torch.argmax(pred,1)
#         age_predictions4.extend(pred.cpu().numpy())
        pred = modelF(images)
        pred = torch.argmax(pred,1)
        age_predictions5.extend(pred.cpu().numpy())
        
        
print("Age predict complite")

Age predict complite


In [34]:
# 2번째 차이를 보자 eval 차이
print(age_predictions1[:10])
print(age_predictions2[:10])
print(age_predictions5[:10])

[1, 1, 1, 1, 0, 0, 1, 1, 1, 0]
[1, 2, 1, 1, 1, 1, 2, 1, 0, 2]
[2, 2, 2, 2, 2, 0, 2, 2, 2, 0]


In [30]:
print(age_predictions1[:10])
print(age_predictions2[:10])
print(age_predictions5[:10])

[1, 1, 1, 1, 0, 0, 1, 1, 1, 0]
[1, 2, 1, 1, 1, 1, 2, 1, 0, 2]
[2, 1, 1, 0, 0, 0, 1, 0, 1, 0]


In [35]:
# 차이를 보자
print(age_predictions2[:30])
print(age_predictions5[:30])

[1, 2, 1, 1, 1, 1, 2, 1, 0, 2, 1, 1, 1, 0, 1, 0, 2, 2, 1, 1, 1, 2, 2, 1, 2, 0, 1, 0, 0, 1]
[2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 0, 0, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1]


In [32]:
print(age_predictions2[:30])
print(age_predictions5[:30])

[1, 2, 1, 1, 1, 1, 2, 1, 0, 2, 1, 1, 1, 0, 1, 0, 2, 2, 1, 1, 1, 2, 2, 1, 2, 0, 1, 0, 0, 1]
[2, 1, 1, 0, 0, 0, 1, 0, 1, 0, 2, 0, 0, 1, 0, 0, 2, 1, 2, 0, 2, 2, 1, 0, 1, 0, 1, 1, 1, 1]


In [15]:
print(age_predictions[:30])

[1, 2, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 2, 1, 2, 1, 1, 2, 1, 0, 1, 0, 1, 1, 1, 1]


In [14]:
print(age_predictions[:30])

[1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0]


In [17]:
test_dir = '/opt/ml/input/data/eval'

# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv('./input/data/eval/info.csv')
loader_age = test_loader2
loader_MaskGender = test_loader

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
modelF = modelF.to(DEVICE)
modelF.eval()

model_MaskGender = model_MaskGender.to(DEVICE)
model_MaskGender.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
# all_predictions = []
age_predictions = []
maskgeder_predictions = []
for images in loader_age:
    with torch.no_grad():
        images = images.to(DEVICE)
        pred = modelF(images)
        pred = torch.argmax(pred,1)
        age_predictions.extend(pred.cpu().numpy())

print("Age predict complite")
        
for images in loader_MaskGender:
    with torch.no_grad():
        images = images.to(DEVICE)
        pred = model_MaskGender(images)
        maskgeder_predictions.extend(pred.cpu().numpy())

print("Mask and Gender predict complite")        

all_predictions = np.array(age_predictions) + np.array(maskgeder_predictions)
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join('./', 'submission.csv'), index=False)
print('test inference is done!')

Age predict complite
Mask and Gender predict complite
test inference is done!


In [18]:
age_predictions[:4]

[2, 2, 2, 2]

In [ ]:
test_dir = '/opt/ml/input/data/eval'

# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv('./input/data/eval/info.csv')
loader = test_loader2

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
modelF = modelF.to(DEVICE)
modelF.eval()


# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
# all_predictions = []
age_predictions1 = []
age_predictions2 = []
age_predictions3 = []
age_predictions4 = []
age_predictions5 = []

for images in loader_age:
    with torch.no_grad():
        images = images.to(DEVICE)
        pred = model_Age(images)
        age_predictions1.extend(pred.cpu().numpy())
        pred = modelB_1(images)
        pred = torch.argmax(pred,1)
        age_predictions2.extend(pred.cpu().numpy())
#         pred = modelD(images)
#         pred = torch.argmax(pred,1)
#         age_predictions3.extend(pred.cpu().numpy())
#         pred = modelE(images)
#         pred = torch.argmax(pred,1)
#         age_predictions4.extend(pred.cpu().numpy())
        pred = modelF(images)
        pred = torch.argmax(pred,1)
        age_predictions5.extend(pred.cpu().numpy())
        
        
print("Age predict complite")